In [1]:
%reload_ext autoreload
%autoreload 2

from semproc.timed_command import TimedCmd
from datetime import datetime
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_, or_, not_
from mpp.models import Response
from mpp.models import BagOfWords

import warnings
warnings.filterwarnings('ignore')

In [13]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [15]:
START = 0
TOTAL = 1  
LIMIT = 1
clauses = [
    Response.format == 'xml',
    not_(or_(
        Response.cleaned_content.startswith("<rdf"),
        Response.cleaned_content.startswith("<RDF")
    ))
]

cmd = 'python ../scripts/word_count_cli.py -x "%(xml)s"'
timeout = 120  # in seconds, more than 2minutes seems like an eternity

In [18]:
for i in xrange(START, TOTAL, LIMIT):
    responses = session.query(Response.id, Response.cleaned_content).filter(
        and_(*clauses)).limit(LIMIT).offset(i).all()

    for response_id, cleaned_content in responses:
        print cleaned_content
        tc = TimedCmd(cmd % {"xml": cleaned_content})
        try:
            status, output, error = tc.run(timeout)
        except:
            print 'failed extraction: ', response_id
            with open('bow_fails.txt', 'a') as f:
                f.write('extract fail: \n'.format(response_id))
            continue

        if error:
            print 'error from cli: ', response_id, error
            with open('bow_fails.txt', 'a') as f:
                f.write('cli fail: \n'.format(response_id))
            continue
            
        print 'output: ', output
        print 'status', status

<oai_dc:dc xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org.libproxy.lib.unc.edu/dc/elements/1.1/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"></oai_dc:dc>
output:  

status 0


In [17]:
error

''

In [30]:
import subprocess

response = session.query(Response).filter(Response.id==626168).first()

cleaned_content = response.cleaned_content

In [31]:


cmd = "python ../scripts/word_count_cli.py -x '%s'" % cleaned_content.replace("'", "\'")

s = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
stdout, stderr = s.communicate()

In [32]:
stdout

'http www isotc211 org gmd md_IDEEu_CHA_2006_es spa http www loc gov standards spa MD_CharacterSetCode_utf8 ISOTC211 MD_CharacterSetCode_utf8 http www isotc211 org resources Codelist gmxCodelists xml#MD_CharacterSetCode dataset http standards iso org ittf PubliclyAvailableStandards ISO_19139_Schemas resources Codelist ML_gmxCodelists xml#MD_ScopeCode dataset Gobierno Vasco Direcci n de Planificaci n Territorial y Urbanismo Servicio Informaci n Territorial geoeuskadi@euskadi eus pointOfContact http standards iso org ittf PubliclyAvailableStandards ISO_19139_Schemas resources Codelist ML_gmxCodelists xml#CI_RoleCode pointOfContact ISO19115 Cor ETRS89-UTM30N EPSG Cartograf a de cambios en el CORINE Land Cover del Pa s Vasco publication http standards iso org ittf PubliclyAvailableStandards ISO_19139_Schemas resources Codelist ML_gmxCodelists xml#CI_DateTypeCode publication CHA_2006_EUSKADI Eusko Jaurlaritza Gobierno Vasco Mapa en formato vectorial que recoge los cambios en la ocupaci n de

In [25]:
stderr

''

In [18]:
cleaned_content

u'<?xml version="1.0" encoding="UTF-8"?> <WMS_Capabilities version="1.3.0" updateSequence="2015-06-25T19:21:05.812Z" xmlns="http://www.opengis.net/wms" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wms http://schemas.opengis.net/wms/1.3.0/capabilities_1_3_0.xsd"> <Service> <Name>WMS</Name> <Title>Coastal and Marine Geology THREDDS Server</Title> <Abstract>Scientific Data</Abstract> <KeywordList> <Keyword>bathymetry</Keyword> <Keyword>lidar</Keyword> <Keyword>topography</Keyword> <Keyword>ocean</Keyword> <Keyword>earth science</Keyword> </KeywordList> <OnlineResource xlink:type="simple" xlink:href="http://stellwagen.er.usgs.gov/"/> <ContactInformation> <ContactPersonPrimary> <ContactPerson>Webmaster</ContactPerson> <ContactOrganization>US Geological Survey</ContactOrganization> </ContactPersonPrimary> <ContactVoiceTelephone>(508) 457-2356</ContactVoiceTelephone> <ContactElectronicMailAddress>em

In [29]:
clauses = [
    Response.format == 'xml',
    not_(or_(
        Response.cleaned_content.startswith("<rdf"),
        Response.cleaned_content.startswith("<RDF")
    ))
]
for i in xrange(1000, 1110, 10):
    responses = session.query(
        Response
    ).filter(
        and_(*clauses)
    ).limit(10).offset(i).all()

    for response in responses:
        print response.id, response.bags_of_words
#         if response.bags_of_words:
#             print 'has bag'
#             continue
#         print 'no bag'

486897 []
486898 []
486900 []
486903 []
486904 []
486906 []
486907 []
486908 []
486909 []
486910 []
488189 []
488190 []
488191 []
488192 []
488194 []
488195 []
488196 []
488198 []
488199 []
488200 []
489586 []
489533 []
489534 []
489535 []
489536 []
489538 []
489539 []
489541 []
494460 []
489543 []
490874 []
490875 []
490877 []
490878 []
490879 []
496348 []
490881 []
490882 []
490884 []
490885 []
492176 []
492177 []
492179 []
492180 []
492181 []
492182 []
492183 []
492188 []
492186 []
492187 []
493546 []
493548 []
493550 []
493552 []
493553 []
493554 []
494091 []
493555 []
493556 []
493558 []
494924 []
494926 []
494927 []
494928 []
494929 []
494930 []
494931 []
494932 []
494933 []
494934 []
496248 [<mpp.models.BagOfWords object at 0x105a3a590>]
496312 [<mpp.models.BagOfWords object at 0x10590a9d0>]
496249 [<mpp.models.BagOfWords object at 0x1059a7b90>]
496250 [<mpp.models.BagOfWords object at 0x105975190>]
496252 [<mpp.models.BagOfWords object at 0x105975f90>]
496253 [<mpp.models.BagOf